#### Importing Dependencies

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import numpy as np
from shutil import copyfile
import matplotlib.pyplot as plt
import glob
import sklearn
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

C:\Users\Tanya Joon\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


#### Reading the Preprocessed CSV file where 1=PD, 0=Control

In [2]:
data=pd.read_csv('Datacnn.csv')
data

,filename,target
0,3123_functional.nii,1
1,3124_functional.nii,1
2,3125_functional.nii,1
3,3132_functional.nii,1
4,3201_1_functional.nii,0
...,...,...
119,4116_3_functional.nii,0
120,4116_4_functional.nii,0
121,4116_5_functional.nii,0
122,4117_functional.nii,1


#### importing data

In [5]:
ff = glob.glob('all_data/*')
len(ff)

124

#### Initializing an array to store the images

In [6]:
images = []
print(images)

[]


#### Loading images
#### Shape Correction (taking middle 30 slides for each scan)
#### appending the images into array

In [7]:
for f in range(len(ff)):
    a = nib.load(ff[f])
    print(ff[f])
    l1,l2,l3=a.shape
    #print(l1,l2,l3)
    a = a.get_data()
    #print(a)
    
    z1= int(l3/2) -15
    z2= int(l3/2) +15
    a = a[:,:,z1:z2]
    #print(a)
    #for i in range (len(ff)):
    images.append((a[:,:,:]))
    #print (a.shape)
#print(images)

all_data\3123_functional.nii
all_data\3124_functional.nii
all_data\3125_functional.nii
all_data\3132_functional.nii
all_data\3201_1_functional.nii
all_data\3201_2_functional.nii
all_data\3201_3_functional.nii
all_data\3201_4_functional.nii
all_data\3220_functional.nii
all_data\3223_functional.nii
all_data\3225_functional.nii
all_data\3228_functional.nii
all_data\3230_functional.nii
all_data\3234_functional.nii
all_data\3235_1_functional.nii
all_data\3235_2_functional.nii
all_data\3237_1_functional.nii
all_data\3237_2_functional.nii
all_data\3237_3_functional.nii
all_data\3237_4_functional.nii
all_data\3409_functional.nii
all_data\3418_functional.nii
all_data\3422_functional.nii
all_data\3423_functional.nii
all_data\3431_functional.nii
all_data\3433_functional.nii
all_data\3435_functional.nii
all_data\3436_functional.nii
all_data\3443_functional.nii
all_data\3444_functional.nii
all_data\3445_functional.nii
all_data\3448_functional.nii
all_data\3453_1_functional.nii
all_data\3453_2_funct

#### Checking Dimension of Image Stack

In [8]:
imagess=np.asarray(images)
imagess.shape

(124, 128, 128, 30)

#### Min Max Normalization

In [9]:
m = np.max(imagess)
mi = np.min(imagess)

In [10]:
print(m, mi)

32767.0 -607.0


In [11]:
images = (images - mi) / (m - mi)

In [12]:
np.min(images), np.max(images)

(0.0, 1.0)

In [13]:
imagess

array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.

#### Reading Output Labels

In [14]:
data=pd.read_csv('Datacnn.csv')
y=data.target
print(y)

0      1
1      1
2      1
3      1
4      0
      ..
119    0
120    0
121    0
122    1
123    1
Name: target, Length: 124, dtype: int64


#### Changing The 4D image Stack to a 2D Stack to feed into Random Forest Classifier

In [15]:
nsamples, nx, ny, nz = imagess.shape
train_dataset = imagess.reshape((nsamples,nx*ny*nz))

#### Training Random Forest Using K=5 fold 

In [37]:
kf = KFold(n_splits=5, random_state=None, shuffle=True) 

for train_index, test_index in kf.split(train_dataset):
    print("Train:", train_index, "Validation:",test_index)
    x_train, x_test = train_dataset[train_index], train_dataset[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    regressor = RandomForestRegressor(n_estimators=20, random_state=0)
    y_train=keras.utils.to_categorical(y_train)
    y_test=keras.utils.to_categorical(y_test)
    regressor.fit(x_train, y_train)
    y_pred = regressor.predict(x_test)
    y_pred_classes=np.argmax(y_pred,axis=1)
    y_true=np.argmax(y_test,axis=1)
    print('prediction', y_pred_classes)
    print('actual    ', y_true)
    count=0
    m=y_true.shape
    m=m[0]
    for i in range(m):
        if y_pred_classes[i]==y_true[i]:
            count=count+1
    accuracy=count/m
    print('accuracy ',accuracy,'\n\n\n\n')

Train: [  0   2   4   5   6   7   8   9  10  13  14  17  18  19  20  21  22  23
  25  26  27  28  30  32  34  35  37  38  39  40  41  43  44  45  46  47
  48  50  51  52  53  55  56  57  58  59  60  61  62  63  64  65  67  68
  70  71  72  73  74  76  77  81  83  84  85  86  87  88  90  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 107 109 110 111 112
 113 115 116 117 118 119 120 122 123] Validation: [  1   3  11  12  15  16  24  29  31  33  36  42  49  54  66  69  75  78
  79  80  82  89 108 114 121]
prediction [0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 1]
actual     [1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0]
accuracy  0.72 




Train: [  0   1   2   3   4   6   7   8  10  11  12  14  15  16  19  20  21  22
  23  24  25  26  28  29  30  31  33  34  35  36  37  38  39  40  42  43
  44  46  48  49  50  52  53  54  55  56  57  58  59  63  64  65  66  68
  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86
  87  88  89  90  91  92  95  

#### Training Without K Fold Cross Validation
#### Training (80%), Testing (20%)

In [20]:
indices = np.arange(124)
x_train,x_test,y_train,y_test, indices_train,indices_test =train_test_split(train_dataset,y,indices,test_size=0.2, random_state=0)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [21]:
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
y_train=keras.utils.to_categorical(y_train)
y_test=keras.utils.to_categorical(y_test)
regressor.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

In [36]:
y_pred_classes=np.argmax(y_pred,axis=1)
y_true=np.argmax(y_test,axis=1)

print('prediction', y_pred_classes)
print('actual    ', y_true)
m=y_true.shape
m=m[0]
# 24 testing Sample
print(m)

prediction [0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1]
actual     [1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0]
24


#### Printing testing Report

In [23]:
map_characters={0: 'Control', 1: 'PD'}

print('\n', sklearn.metrics.classification_report(np.where(y_test > 0)[1], np.argmax(y_pred, axis=1), target_names=list(map_characters.values())), sep='') 


              precision    recall  f1-score   support

     Control       0.82      0.93      0.87        15
          PD       0.88      0.70      0.78        10

    accuracy                           0.84        25
   macro avg       0.85      0.82      0.83        25
weighted avg       0.84      0.84      0.84        25

